**대규모 텐서플로 모델 훈련과 배포**

# 설정

In [1]:
from tensorflow import config

config.list_physical_devices('GPU')

2025-07-29 14:47:28.697021: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753800448.714065    8729 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753800448.721934    8729 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753800448.740699    8729 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753800448.740710    8729 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753800448.740712    8729 computation_placer.cc:177] computation placer alr

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# 텐서플로 모델 서빙하기

먼저 TF 서빙을 사용하여 모델을 배포한 다음 Google Vertex AI에 배포해 본다.

## 텐서플로 서빙 사용

가장 먼저 모델을 빌드하고 학습한 다음 SavedModel 포맷으로 내보낸다.

### SavedModel 내보내기

MNIST 데이터 세트를 로드하고, 스케일을 조정하고, 분할한다.

In [2]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend, layers
from tensorflow import keras
import tensorflow as tf
from pathlib import Path

mnist_ = mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist_
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
backend.clear_session()
model = keras.Sequential()
model.add(keras.Input((28, 28), dtype=tf.uint8))
model.add(layers.Flatten())
model.add(layers.Rescaling(1 / 255))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
model_name = 'my_mnist_model'
model_version = '0001'
model_path = Path(model_name) / model_version
model.export(model_path)

I0000 00:00:1753800452.108598    8729 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5557 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti, pci bus id: 0000:06:00.0, compute capability: 8.6


Epoch 1/10


I0000 00:00:1753800453.346015    8786 service.cc:152] XLA service 0x7fcdb8006b70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753800453.346061    8786 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
2025-07-29 14:47:33.358979: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1753800453.392994    8786 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-07-29 14:47:34.062759: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36_0', 92 bytes spill stores, 96 bytes spill loads

2025-07-29 14:47:34.215710: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36

  76/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2005 - loss: 2.2267

I0000 00:00:1753800455.970206    8786 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7145 - loss: 1.0954

2025-07-29 14:47:43.531830: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 8 bytes spill stores, 8 bytes spill loads

2025-07-29 14:47:43.552289: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 296 bytes spill stores, 296 bytes spill loads

2025-07-29 14:47:43.703566: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 8 bytes spill stores, 8 bytes spill loads

2025-07-29 14:47:43.830859: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 792 bytes spill stores, 844 bytes spill loads

2025-07-29 14:47:43.956906: I external/local_xla

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.7146 - loss: 1.0951 - val_accuracy: 0.9026 - val_loss: 0.3752
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8997 - loss: 0.3683 - val_accuracy: 0.9196 - val_loss: 0.2979
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9125 - loss: 0.3074 - val_accuracy: 0.9276 - val_loss: 0.2620
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9236 - loss: 0.2709 - val_accuracy: 0.9342 - val_loss: 0.2395
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9294 - loss: 0.2526 - val_accuracy: 0.9374 - val_loss: 0.2228
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9361 - loss: 0.2290 - val_accuracy: 0.9430 - val_loss: 0.2088
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9398 - loss: 0.2160 - val_accuracy: 0.9472 - val_loss: 0.1950
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9443 - loss: 0.1996 - val_accura

INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


Saved artifact at 'my_mnist_model/0001'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.uint8, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  140525231223632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140525231224784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140525231222096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140525231225360: TensorSpec(shape=(), dtype=tf.resource, name=None)


파일 트리를 살핀다:

In [3]:
sorted([str(path) for path in model_path.parent.glob('**/*')])

['my_mnist_model/0001',
 'my_mnist_model/0001/assets',
 'my_mnist_model/0001/fingerprint.pb',
 'my_mnist_model/0001/saved_model.pb',
 'my_mnist_model/0001/variables',
 'my_mnist_model/0001/variables/variables.data-00000-of-00001',
 'my_mnist_model/0001/variables/variables.index']

SavedModel을 검사한다:

In [4]:
!saved_model_cli show --dir '{model_path}'

2025-07-29 14:48:23.552398: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753800503.566079    9601 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753800503.570123    9601 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753800503.581844    9601 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753800503.581881    9601 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753800503.581887    9601 computation_placer.cc:177] computation placer alr

In [5]:
!saved_model_cli show --dir '{model_path}' --tag_set serve

2025-07-29 14:48:26.212781: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753800506.225814    9617 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753800506.229860    9617 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753800506.241359    9617 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753800506.241394    9617 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753800506.241400    9617 computation_placer.cc:177] computation placer alr

In [6]:
!saved_model_cli show --dir '{model_path}' --tag_set serve --signature_def serving_default

2025-07-29 14:48:28.714176: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753800508.727995    9633 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753800508.732280    9633 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753800508.744008    9633 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753800508.744052    9633 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753800508.744058    9633 computation_placer.cc:177] computation placer alr

더 자세한 내용을 보려면 다음 명령을 실행한다:
```ipython
!saved_model_cli show --dir '{model_path}' --all
```

### 텐서플로 서빙 설치 및 시작하기

도커를 사용해 TF 서빙을 설치하려면 먼저 [Docker](https://docs.docker.com/install/)가 설치되어 있는지 확인한 후 터미널에서 다음 명령을 실행한다.
```bash
docker pull tensorflow/serving:latest-gpu

docker run -e MODEL_NAME=my_mnist_model --gpus all -it -p 8500:8500 -p 8501:8501 -v path\to\my_mnist_model:/models/my_mnist_model tensorflow/serving:latest-gpu
```

### REST API로 TF 서빙에 쿼리하기

다음으로 TF 서빙에 REST 쿼리를 전송한다:

In [7]:
import json

X_new = X_test[:3]  # 분류할 새로운 숫자 이미지가 3개 있다고 가정한다.
request_json = json.dumps({'signature_name': 'serving_default', 'instances': X_new.tolist()})

In [8]:
request_json[:100] + '...' + request_json[-10:]

'{"signature_name": "serving_default", "instances": [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..., 0, 0]]]}'

이제 텐서플로 서빙의 REST API를 사용하여 예측한다:

In [9]:
import requests

server_url = 'http://172.17.0.2:8501/v1/models/my_mnist_model:predict'
response = requests.post(server_url, request_json)
response.raise_for_status()  # 오류 발생 시 예외 발생
response = response.json()

In [10]:
import numpy as np

y_proba = np.array(response['predictions'])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.98, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.98, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

### gRPC API로 TF 서빙에 쿼리하기

In [11]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest

request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = 'serving_default'
input_name = 'keras_tensor'
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

In [12]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel('172.17.0.2:8500')
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
response = predict_service.Predict(request, timeout=10.0)

응답을 텐서로 변환한다:

In [13]:
output_name = 'output_0'
outputs_proto = response.outputs[output_name]
y_proba = tf.make_ndarray(outputs_proto)

In [14]:
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.98, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.98, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)